In [11]:
import pandas as pd
from utils import create_mmsi_dict_from_file

In [13]:
file_name = "data/mmsi_type.txt"
mmsi_map = create_mmsi_dict_from_file(file_name)


if mmsi_map:
    print("--- Successfully created dictionary ---")

--- Successfully created dictionary ---


In [ ]:
df = pd.read_csv("data/ais_combined.csv")
df.head()

,MMSI,SOG,COG,Longtitude,Latitude,Timestamp,Segment
0,200000000,6.739216,90.1,11.591552,56.123522,2025-02-27 00:00:05,0
1,200000000,6.739216,90.0,11.591770,56.123522,2025-02-27 00:00:06,0
2,200000000,6.739216,90.0,11.591770,56.123522,2025-02-27 00:00:07,0
3,200000000,6.739216,89.8,11.592530,56.123518,2025-02-27 00:00:13,0
4,200000000,6.739216,89.7,11.592637,56.123518,2025-02-27 00:00:14,0


In [18]:
df_with_types = df.copy()
df_with_types['Type'] = df_with_types['MMSI'].astype(str).map(mmsi_map)
df_with_types.head()

,MMSI,SOG,COG,Longtitude,Latitude,Timestamp,Segment,Type
0,200000000,6.739216,90.1,11.591552,56.123522,2025-02-27 00:00:05,0,Tanker
1,200000000,6.739216,90.0,11.591770,56.123522,2025-02-27 00:00:06,0,Tanker
2,200000000,6.739216,90.0,11.591770,56.123522,2025-02-27 00:00:07,0,Tanker
3,200000000,6.739216,89.8,11.592530,56.123518,2025-02-27 00:00:13,0,Tanker
4,200000000,6.739216,89.7,11.592637,56.123518,2025-02-27 00:00:14,0,Tanker


In [20]:
unique_mmsi = df['MMSI'].unique()
unique_types = df_with_types['Type'].unique()

print("Total unique MMSI count:", len(unique_mmsi))
print("Unique ship types in dataset:", unique_types)

Total unique MMSI count: 991
Unique ship types in dataset: ['Tanker' 'Unknown' 'Sailing vessel' 'Other type' 'Cargo ship'
 'Passenger ship' 'Tanker (HAZ-D)' 'Tanker (HAZ-A)' 'Cargo ship (HAZ-A)'
 'Tanker (HAZ-B)' 'Fishing vessel' 'Law enforcment' 'Pilot' 'Military ops'
 'Dredging or UW ops' 'Tug' 'Anti-polution' 'HSC' 'SAR'
 'Ship type not found' 'Pleasure craft' 'Towing vessel' 'Port tender'
 'Diving ops' 'Cargo ship (HAZ-D)' 'Passenger ship (HAZ-A)'
 'Other type (HAZ-A)' 'Other type (HAZ-B)' 'Cargo ship (HAZ-B)'
 'Tanker (HAZ-C)']


In [22]:
allowed_type = ['Cargo ship', 'Cargo ship (HAZ-A)', 'Cargo ship (HAZ-B)']
df_cargo = df_with_types[df_with_types['Type'].isin(allowed_type)]
df_cargo.head()

,MMSI,SOG,COG,Longtitude,Latitude,Timestamp,Segment,Type
33012,209056000,5.504551,286.9,14.087317,54.677550,2025-02-27 00:00:08,0,Cargo ship
33013,209056000,5.504551,287.0,14.086550,54.677682,2025-02-27 00:00:17,0,Cargo ship
33014,209056000,5.504551,287.0,14.085700,54.677825,2025-02-27 00:00:27,0,Cargo ship
33015,209056000,5.504551,287.1,14.084912,54.677972,2025-02-27 00:00:38,0,Cargo ship
33016,209056000,5.504551,287.1,14.084072,54.678125,2025-02-27 00:00:47,0,Cargo ship
